<img src="http://bioinfo-pharma.u-strasbg.fr/labwebsite/images/logo_UMR7200.png" width="250" align="right" style="height:240px">


# Strasbourg Summer School in Chemoinformatics 2024 - Multistate modelling using LIT-AlphaFold

<img src="https://infochim.chimie.unistra.fr/squelettes-infochim/images/logo-conferences.png" width="200" align="right" style="height:240px">

In this tutorial we will present you AlphaFold and its use, as well as specific application aimed at modelling different conformational states of a target protein.

All data and information are available from this tutorial [GitHub page](https://github.com/LIT-CCM-lab/CS3_2024_AlphaFold_Tutorial).

##  Environment setup

In [ ]:
# @title Install dependencies { display-mode: "form" }
import os
print("Installing LIT-AlphaFold")
if not os.path.isfile("CONDA_READY"):
    os.system("pip install --quiet condacolab")
    import condacolab
    condacolab.install()
    os.system("mamba install --quiet -c conda-forge -c bioconda python=3.10 openmm==7.7.0 pdbfixer kalign2=2.04 hhsuite=3.3.0 mmseqs2=14.7e284  polyleven") 
    os.system("touch CONDA_READY")
print("conda installed")

if not os.path.isfile("COLABFOLD_READY"):
    os.system("pip install --quiet alphapulldown==0.40.4 --no-deps")
    os.system('pip install --quiet --no-warn-conflicts "colabfold[alphafold-minus-jax] @ git+https://github.com/sokrypton/ColabFold@v1.5.3"')
    os.system("pip install --quiet https://storage.googleapis.com/jax-releases/cuda11/jaxlib-0.3.25+cuda11.cudnn82-cp310-cp310-manylinux2014_x86_64.whl")
    os.system("pip install --quiet jax==0.3.25 chex==0.1.6 biopython==1.80")
    os.system("touch COLABFOLD_READY")
print("Environment ready")

if not os.path.isfile("LITAF_READY"):
    os.system("git clone -q https://github.com/LIT-CCM-lab/LIT-AlphaFold")
    os.system("pip install --quiet LIT-AlphaFold/litaf/ ")
    os.system("touch LITAF_READY")
print("LIT-AlphaFold installed")

if not os.path.isfile('DATA_READY'):
    os.system("git clone -q https://github.com/LIT-CCM-lab/CS3_2024_AlphaFold_Tutorial")
    os.system("touch DATA_READY")
print("Tutorial material downloaded")

import jax
try:
    # check if TPU is available
    import jax.tools.colab_tpu
    jax.tools.colab_tpu.setup_tpu()
    print('Running on TPU')
except:
    if jax.local_devices()[0].platform == 'cpu':
        print("WARNING: no GPU detected, will be using CPU")
    else:
        import tensorflow as tf
        print('Running on GPU')

In [ ]:
# @title Set local variables { display-mode: "form" }
github = 'CS3_2024_AlphaFold_Tutorial/'
protein_structures = f'{github}protein_structures'
protein_features = f'{github}protein_features'
reference_structures = f'{github}reference_structures'
inputs = f'{github}inputs'

import warnings
warnings.filterwarnings('ignore')
from google.colab import files
from CS3_2024_AlphaFold_Tutorial.utils import align_on_ref, show_aligned, plot_chain_legend

In [ ]:
# @title Javascript test { display-mode: "form" }
test_file = f"{protein_structures}/1ezg.pdb"
view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js', width = 800, height = 600)
view.addModel(open(test_file,'r').read(),'pdb').show()

## Example system - Segment of a viral polyprotein
Precalculated example of the results obtained using AlphaFold

In [ ]:
# @title  Display the MSA  { display-mode: "form" }
import pickle
import bz2
import matplotlib.pyplot as plt
from colabfold.plot import plot_msa_v2

polyprot = pickle.load(bz2.BZ2File(f'{protein_features}/viral_polyprotein.pkl.bz2', "rb"))

%matplotlib inline
plot_msa_v2(polyprot.feature_dict)
plt.show()
plt.close()

In [ ]:
# @title Display 3D structure { display-mode: "form" }
#from litaf.utils import show_pdb
from colabfold.colabfold import plot_plddt_legend

from litaf.utils import show_pdb

rank_num = 1 #@param ["1", "2", "3", "4", "5"] {type:"raw"}
color = "lDDT" #@param ["chain", "lDDT", "rainbow"]
show_sidechains = False #@param {type:"boolean"}
show_mainchains = False #@param {type:"boolean"}
tag = rank_num - 1

#@markdown Click on any atom to show a label with the residue name, the residue number, and the plDDT value.<br/>Click a second time on the same atom to hide the label.


vprot_path = "viral_polyprotein"

pdb_filename = f"{protein_structures}/{vprot_path}/ranked_{tag}.pdb"

show_pdb(pdb_filename, 1, show_sidechains, show_mainchains, color).show()
if color == "lDDT":
  plot_plddt_legend().show()

In [ ]:
# @title Plot plDDT & PAE { display-mode: "form" }
from colabfold.colabfold import plot_plddts, plot_paes
import json
import pickle

with open(f"{protein_structures}/{vprot_path}/ranking_debug.json", 'r') as f:
    data = json.load(f)

plddts = []
paes = []
for file in data['order']:
    pkl_f = bz2.BZ2File(f"{protein_structures}/{vprot_path}/result_{file}.pkl.bz2", "r")
    model_data = pickle.load(pkl_f)
    plddts.append(model_data['plddt'])
    paes.append(model_data['predicted_aligned_error'])

plddts_plot = plot_plddts(plddts, [len(polyprot.sequence)])
pae_plot = plot_paes(paes, [len(polyprot.sequence)])

plddts_plot.savefig(f"{protein_structures}/{vprot_path}/plddts_plot.png")
pae_plot.savefig(f"{protein_structures}/{vprot_path}/pae_plots.png")

## SARS-CoV-2 3CL-like proteinase
We run full calculations for this one, from using MMseqs2 to running all five models

In [ ]:
# @title Input generation { display-mode: "form" }

from pathlib import Path
import logging
from absl import logging as absl_logging
import os
from alphapulldown.utils import parse_fasta
from litaf.objects import MonomericObjectMmseqs2
from colabfold.utils import DEFAULT_API_SERVER

sars_cov_path = '3CL-proteinase'

for handler in logging.root.handlers[:]:
        logging.root.removeHandler(handler)

logging.basicConfig(format="%(levelname)s - %(message)s",
                    level = logging.INFO,
                    handlers=[
                    logging.FileHandler(f"{protein_features}/sars-cov2.log"),
                    logging.StreamHandler(),
                    ])
absl_logging.set_verbosity(absl_logging.ERROR)

with open(f'{inputs}/sars-cov2.fasta') as f:
    seq, description = parse_fasta(f.read())


sarscov_monomer = MonomericObjectMmseqs2(description[0], seq[0])
sarscov_monomer.make_features(DEFAULT_API_SERVER=DEFAULT_API_SERVER,
                                            output_dir=protein_features,
                                            templates_path='mmseqs2',
                                            max_template_date='2050-01-01',
                                            filter_t= [{}, []])

%matplotlib inline
plot_msa_v2(sarscov_monomer.feature_dict)
plt.show()
plt.close()

In [ ]:
from litaf.create_input import create_interactors_colab

# @title Calculations settings { display-mode: "form" }
#@markdown **Prediction parameters**

only_template_models = False # @param {type: 'boolean'}
use_templates = True # @param {type: 'boolean'}
    
from litaf.utils import create_colabfold_runners
from colabfold.download import download_alphafold_params

logging.basicConfig(format="%(levelname)s - %(message)s",
                    level = logging.INFO,
                    handlers=[
                    logging.FileHandler(f"{protein_structures}/sars-cov2.log"),
                    logging.StreamHandler(),
                    ])

n = 2 if only_template_models else 5
if not all([os.path.isfile(f"/params/params_model_{i}_ptm.npz") for i in range(1,6)]):
    download_alphafold_params(f'alphafold2_ptm', Path("."))
model_runners = create_colabfold_runners(
                    '_ptm',
                    n,
                    use_templates,
                    5,
                    ".",
                    None,
                    None,
                    1,
                    False,
                    False,
                    False)

In [ ]:
# @title Run predictions { display-mode: "form" }
from litaf.predict_structure import predict, ModelsToRelax

output_path = os.path.join(protein_structures, sarscov_monomer.description)
Path(output_path).mkdir(parents=True, exist_ok=True)
predict(
        model_runners,
        output_path,
        sarscov_monomer.feature_dict,
        0,
        False,
        fasta_name=sarscov_monomer.description,
        models_to_relax=ModelsToRelax.NONE,
        seqs=[sarscov_monomer.sequence],
        allow_resume=True
    )

In [ ]:
# @title Display 3D structure { display-mode: "form" }
rank_num = 1 #@param ["1", "2", "3", "4", "5"] {type:"raw"}
color = "lDDT" #@param ["chain", "lDDT", "rainbow"]
show_sidechains = False #@param {type:"boolean"}
show_mainchains = False #@param {type:"boolean"}
tag = rank_num - 1

pdb_filename = f"{output_path}/ranked_{tag}.pdb"

show_pdb(pdb_filename, 1, show_sidechains, show_mainchains, color).show()
if color == "lDDT":
  plot_plddt_legend().show()

In [ ]:
# @title Plot plDDT & PAE { display-mode: "form" }
with open(f"{output_path}/ranking_debug.json", 'r') as f:
    data = json.load(f)

plddts = []
paes = []
for file in data['order']:
    pkl_f = bz2.BZ2File(f"{protein_structures}/{sars_cov_path}/result_{file}.pkl.bz2", "r")
    model_data = pickle.load(pkl_f)
    plddts.append(model_data['plddt'])
    paes.append(model_data['predicted_aligned_error'])

plddts_plot = plot_plddts(plddts, [len(sarscov_monomer.sequence)])
pae_plot = plot_paes(paes, [len(sarscov_monomer.sequence)])

plddts_plot.savefig(f"{output_path}/plddts_plot.png")
pae_plot.savefig(f"{output_path}/pae_plots.png")

In [ ]:
# @title Align AlphaFold models on reference structures { display-mode: "form" }
ref_b_file = f'{reference_structures}/6XQS.pdb'
ref_b_aligned = f'{reference_structures}/6XQS_aligned.pdb'
ref_a_file = f'{reference_structures}/6WQF.pdb'
ref_a_aligned = f'{reference_structures}/6WQF_aligned.pdb'

align_on_ref(ref_a_file, ref_b_file, 'SV6')
for i in range(5):
    align_on_ref(f"{output_path}/ranked_{i}.pdb", ref_b_file, 'SV6')

In [ ]:
# @title Comparison between the free protein (PDBID: 6WQF) and the ligand bound protein (PDBID: 6XQS)  { display-mode: "form" }

holo_cartoon_color = "green" #@param ["green", "cyan", "magenta", "purple", "white", "orange", "yellow", "blue"]
apo_cartoon_color = "orange" #@param ["green", "cyan", "magenta", "purple", "white", "orange", "yellow", "blue"]
ligand_color = "whiteCarbon" #@param ["greenCarbon", "cyanCarbon", "magentaCarbon", "purpleCarbon", "whiteCarbon", "orangeCarbon", "yellowCarbon", "blueCarbon"]
holo_sidechain_color = "cyanCarbon" #@param ["greenCarbon", "cyanCarbon", "magentaCarbon", "purpleCarbon", "whiteCarbon", "orangeCarbon", "yellowCarbon", "blueCarbon"]
apo_sidechain_color = "yellowCarbon" #@param ["greenCarbon", "cyanCarbon", "magentaCarbon", "purpleCarbon", "whiteCarbon", "orangeCarbon", "yellowCarbon", "blueCarbon"]

#@markdown Click on any atom to show a label with the residue name, the residue number, and the atom name.<br/>Click a second time on the same atom to hide the label.

show_aligned(ref_a_aligned, ref_b_aligned, 'SV6',
             model_cartoon_color = apo_cartoon_color,
             ref_cartoon_color = holo_cartoon_color,
             ligand_color = ligand_color,
             model_sidecahin_color = apo_sidechain_color,
             ref_sidecahin_color = holo_sidechain_color).show()
plot_chain_legend(name_model = 'Apo', name_ref = 'Holocd ',
             model_cartoon_color = apo_cartoon_color,
             ref_cartoon_color = holo_cartoon_color,
             ligand_color = ligand_color[:-6],
             model_sidecahin_color = apo_sidechain_color[:-6],
             ref_sidecahin_color = holo_sidechain_color[:-6]).show()

In [ ]:
# @title Comparison between the AlphaFold model and the ligand bound protein (PDBID: 6XQS)  { display-mode: "form" }
rank_num = 1 #@param ["1", "2", "3", "4", "5"] {type:"raw"}

reference_cartoon_color = "green" #@param ["green", "cyan", "magenta", "purple", "white", "orange", "yellow", "blue"]
AF_model_cartoon_color = "magenta" #@param ["green", "cyan", "magenta", "purple", "white", "orange", "yellow", "blue"]
ligand_color = "whiteCarbon" #@param ["greenCarbon", "cyanCarbon", "magentaCarbon", "purpleCarbon", "whiteCarbon", "orangeCarbon", "yellowCarbon", "blueCarbon"]
reference_sidechain_color = "cyanCarbon" #@param ["greenCarbon", "cyanCarbon", "magentaCarbon", "purpleCarbon", "whiteCarbon", "orangeCarbon", "yellowCarbon", "blueCarbon"]
AF_model_sidechain_color = "purpleCarbon" #@param ["greenCarbon", "cyanCarbon", "magentaCarbon", "purpleCarbon", "whiteCarbon", "orangeCarbon", "yellowCarbon", "blueCarbon"]



aligned_model = f"{output_path}/ranked_{rank_num - 1}_aligned.pdb"
show_aligned(aligned_model, ref_b_aligned, 'SV6',
             model_cartoon_color = AF_model_cartoon_color,
             ref_cartoon_color = reference_cartoon_color,
             ligand_color = ligand_color,
             model_sidecahin_color = AF_model_sidechain_color,
             ref_sidecahin_color = reference_sidechain_color).show()
plot_chain_legend(model_cartoon_color = AF_model_cartoon_color,
             ref_cartoon_color = reference_cartoon_color,
             ligand_color = ligand_color[:-6],
             model_sidecahin_color = AF_model_sidechain_color[:-6],
             ref_sidecahin_color = reference_sidechain_color[:-6]).show()

In [ ]:
# @title Comparison between the AlphaFold model and the free protein (PDBID: 6WQF) { display-mode: "form" }
rank_num = 1 #@param ["1", "2", "3", "4", "5"] {type:"raw"}

reference_cartoon_color = "green" #@param ["green", "cyan", "magenta", "purple", "white", "orange", "yellow", "blue"]
AF_model_cartoon_color = "magenta" #@param ["green", "cyan", "magenta", "purple", "white", "orange", "yellow", "blue"]
ligand_color = "whiteCarbon" #@param ["greenCarbon", "cyanCarbon", "magentaCarbon", "purpleCarbon", "whiteCarbon", "orangeCarbon", "yellowCarbon", "blueCarbon"]
reference_sidechain_color = "cyanCarbon" #@param ["greenCarbon", "cyanCarbon", "magentaCarbon", "purpleCarbon", "whiteCarbon", "orangeCarbon", "yellowCarbon", "blueCarbon"]
AF_model_sidechain_color = "purpleCarbon" #@param ["greenCarbon", "cyanCarbon", "magentaCarbon", "purpleCarbon", "whiteCarbon", "orangeCarbon", "yellowCarbon", "blueCarbon"]

aligned_model = f"{output_path}/ranked_{rank_num - 1}_aligned.pdb"

show_aligned(aligned_model, ref_a_aligned, 'GST',
             model_cartoon_color = AF_model_cartoon_color,
             ref_cartoon_color = reference_cartoon_color,
             ligand_color = ligand_color,
             model_sidecahin_color = AF_model_sidechain_color,
             ref_sidecahin_color = reference_sidechain_color).show()
plot_chain_legend(model_cartoon_color = AF_model_cartoon_color,
             ref_cartoon_color = reference_cartoon_color,
             ligand_color = ligand_color[:-6],
             model_sidecahin_color = AF_model_sidechain_color[:-6],
             ref_sidecahin_color = reference_sidechain_color[:-6]).show()

In [ ]:
# @title Package and Download results { display-mode: "form" }
results_zip = f"3CL-proteinase_result.zip"
os.system(f"zip -r {results_zip} {output_path} {github}/protein_features/3CL-proteinase.pkl.bz2")
files.download(results_zip)

## Multistate modelling - $\beta$2 adrenergic receptor

### Default calculation

GPCRs are characterized by assuming two main conformational states, an active state and an ianctive state. The affinity of a ligand for the receptor depends on its conformational state. Agonist ligands have a higher affinity for active state conformations, antagonists have a higher affinity for inactive state confomrations.

AlphaFold generates for class A GPCRs only conformations in the inactive state. As an example we report the structure of ADRB2, to avoid any bias we removed all ADRB2 structures from the used tempaltes.

In [ ]:
# @title Display 3D structure { display-mode: "form" }
rank_num = 1 #@param ["1", "2", 3, 4, 5] {type:"raw"}
color = "lDDT" #@param ["chain", "lDDT", "rainbow"]
show_sidechains = False #@param {type:"boolean"}
show_mainchains = False #@param {type:"boolean"}
tag = rank_num - 1

adrb2_path = 'ADRB2_filter_adrb2'

pdb_filename = f"{protein_structures}/{adrb2_path}/ranked_{tag}.pdb"

show_pdb(pdb_filename, 1, show_sidechains, show_mainchains, color).show()
if color == "lDDT":
  plot_plddt_legend().show()

In [ ]:
# @title Plot plDDT & PAE { display-mode: "form" }
from litaf.objects import load_monomer_objects
adrb2_monomer = load_monomer_objects({'ADRB2_filter_adrb2': protein_features}, 'ADRB2_filter_adrb2')

with open(f"{protein_structures}/{adrb2_path}/ranking_debug.json", 'r') as f:
    data = json.load(f)

plddts = []
paes = []
for file in data['order']:
    pkl_f = bz2.BZ2File(f"{protein_structures}/{adrb2_path}/result_{file}.pkl.bz2", "r")
    model_data = pickle.load(pkl_f)
    plddts.append(model_data['plddt'])
    paes.append(model_data['predicted_aligned_error'])

plddts_plot = plot_plddts(plddts, [len(adrb2_monomer.sequence)])
pae_plot = plot_paes(paes, [len(adrb2_monomer.sequence)])

In [ ]:
# @title Comparison between the AlphaFold structure and inactive state ADRB2 (PDBID: 4GBR)  { display-mode: "form" }
rank_num = 1 #@param ["1", "2", "3", "4", "5"] {type:"raw"}
tag = rank_num - 1

reference_cartoon_color = "orange" #@param ["green", "cyan", "magenta", "purple", "white", "orange", "yellow", "blue"]
AF_model_cartoon_color = "magenta" #@param ["green", "cyan", "magenta", "purple", "white", "orange", "yellow", "blue"]
ligand_color = "whiteCarbon" #@param ["greenCarbon", "cyanCarbon", "magentaCarbon", "purpleCarbon", "whiteCarbon", "orangeCarbon", "yellowCarbon", "blueCarbon"]
reference_sidechain_color = "yellowCarbon" #@param ["greenCarbon", "cyanCarbon", "magentaCarbon", "purpleCarbon", "whiteCarbon", "orangeCarbon", "yellowCarbon", "blueCarbon"]
AF_model_sidechain_color = "purpleCarbon" #@param ["greenCarbon", "cyanCarbon", "magentaCarbon", "purpleCarbon", "whiteCarbon", "orangeCarbon", "yellowCarbon", "blueCarbon"]

adrb2_path = 'ADRB2_filter_adrb2'

pdb_filename = f"{protein_structures}/{adrb2_path}/ranked_{tag}.pdb"

ref_file = f'{reference_structures}/4GBR.pdb'
ref_aligned = f'{reference_structures}/4GBR_aligned.pdb'
aligned_model = pdb_filename.split('.')[0]+'_aligned.pdb'

align_on_ref(pdb_filename, ref_file, 'CAU')
show_aligned(aligned_model, ref_aligned, 'CAU',
             model_cartoon_color = AF_model_cartoon_color,
             ref_cartoon_color = reference_cartoon_color,
             ligand_color = ligand_color,
             model_sidecahin_color = AF_model_sidechain_color,
             ref_sidecahin_color = reference_sidechain_color).show()
plot_chain_legend(model_cartoon_color = AF_model_cartoon_color,
             ref_cartoon_color = reference_cartoon_color,
             ligand_color = ligand_color[:-6],
             model_sidecahin_color = AF_model_sidechain_color[:-6],
             ref_sidecahin_color = reference_sidechain_color[:-6]).show()

In [ ]:
# @title Comparison between the AlphaFold structure and active state ADRB2 (PDBID: 3P0G)  { display-mode: "form" }
rank_num = 1 #@param ["1", "2", "3", "4", "5"] {type:"raw"}
tag = rank_num - 1

reference_cartoon_color = "green" #@param ["green", "cyan", "magenta", "purple", "white", "orange", "yellow", "blue"]
AF_model_cartoon_color = "magenta" #@param ["green", "cyan", "magenta", "purple", "white", "orange", "yellow", "blue"]
ligand_color = "whiteCarbon" #@param ["greenCarbon", "cyanCarbon", "magentaCarbon", "purpleCarbon", "whiteCarbon", "orangeCarbon", "yellowCarbon", "blueCarbon"]
reference_sidechain_color = "cyanCarbon" #@param ["greenCarbon", "cyanCarbon", "magentaCarbon", "purpleCarbon", "whiteCarbon", "orangeCarbon", "yellowCarbon", "blueCarbon"]
AF_model_sidechain_color = "purpleCarbon" #@param ["greenCarbon", "cyanCarbon", "magentaCarbon", "purpleCarbon", "whiteCarbon", "orangeCarbon", "yellowCarbon", "blueCarbon"]

pdb_filename = f"{protein_structures}/{adrb2_path}/ranked_{tag}.pdb"

ref_file = f'{reference_structures}/3P0G.pdb'
ref_aligned = f'{reference_structures}/3P0G_aligned.pdb'
aligned_model = pdb_filename.split('.')[0]+'_aligned.pdb'

align_on_ref(pdb_filename, ref_file, 'P0G')
show_aligned(aligned_model, ref_aligned, 'P0G',
             model_cartoon_color = AF_model_cartoon_color,
             ref_cartoon_color = reference_cartoon_color,
             ligand_color = ligand_color,
             model_sidecahin_color = AF_model_sidechain_color,
             ref_sidecahin_color = reference_sidechain_color).show()
plot_chain_legend(model_cartoon_color = AF_model_cartoon_color,
             ref_cartoon_color = reference_cartoon_color,
             ligand_color = ligand_color[:-6],
             model_sidecahin_color = AF_model_sidechain_color[:-6],
             ref_sidecahin_color = reference_sidechain_color[:-6]).show()

### Tempalte selection

Since GPCRs are a widely studied family of targets, information about their conformational state is available in GPCRdb, an online resource containing structural annotation.

LIT-AlphaFold can be used to query directly this database, in order to select automatically structures in the active or inactive state, without the need for manual selection.

In this case we select only templates of GPCRs not corresponding to human ADRB2 in an active state conformation.

For more information about template selection please consult the [LIT-AlphaFold wiki](https://github.com/LIT-CCM-lab/LIT-AlphaFold/wiki) and [GPCRdb]().

In [ ]:
# @title Load query { display-mode: "form" }
import yaml
from litaf.filterpdb import load_template_filter

logging.basicConfig(format="%(levelname)s - %(message)s",
                    level = logging.INFO,
                    handlers=[
                    logging.FileHandler(f"{protein_features}/adrb2.log"),
                    logging.StreamHandler(),
                    ])

active_state_filter = load_template_filter(f'{inputs}/active_state.yaml')
print("Content of the query:\n")
print(yaml.dump(active_state_filter[0]))

In [ ]:
# @title Querying GPCRdb { display-mode: "form" }
active_feat_dict = adrb2_monomer.filter_templates(active_state_filter)

### AF2_GPCR_Kinase
We perform calculations using the method developed by [Sala et al.](https://www.frontiersin.org/articles/10.3389/fmolb.2023.1121962/full).

In [ ]:
# @title Calculations settings { display-mode: "form" }
only_template_models = True # @param {type: 'boolean'}
use_templates = True # @param {type: 'boolean'}
max_seqs = "8:16" # @param ["8:16", "16:32"]

max_seq, max_extra_seq = max_seqs.split(':')
max_seq = int(max_seq)
max_extra_seq = int(max_extra_seq)
run_description = f'_MSA-subsampling-{max_seqs}'

from litaf.utils import read_custom
from litaf.objects import MultimericObject
    
logging.basicConfig(format="%(levelname)s - %(message)s",
                    level = logging.INFO,
                    handlers=[
                    logging.FileHandler(f"{protein_structures}/adrb2.log"),
                    logging.StreamHandler(),
                    ])

n = 2 if only_template_models else 5
if not all([os.path.isfile(f"/params/params_model_{i}_ptm.npz") for i in range(1,6)]):
    download_alphafold_params(f'alphafold2_ptm', Path("."))
model_runners = create_colabfold_runners(
                    '_ptm',
                    n,
                    use_templates,
                    5,
                    ".",
                    max_seq,
                    max_extra_seq,
                    1,
                    False,
                    False,
                    False)

In [ ]:
# @title Run predictions { display-mode: "form" }
output_path = os.path.join(protein_structures, adrb2_monomer.description+run_description)
Path(output_path).mkdir(parents=True, exist_ok=True)
adrb2_monomer.input_seqs = [adrb2_monomer.sequence]
predict(
        model_runners,
        output_path,
        active_feat_dict,
        0,
        False,
        fasta_name=adrb2_monomer.description+run_description,
        models_to_relax=ModelsToRelax.NONE,
        seqs=[adrb2_monomer.sequence],
        allow_resume=True
    )

In [ ]:
# @title Display 3D structure { display-mode: "form" }
rank_num = 1 #@param ["1", "2"] {type:"raw"}
color = "lDDT" #@param ["chain", "lDDT", "rainbow"]
show_sidechains = False #@param {type:"boolean"}
show_mainchains = False #@param {type:"boolean"}
tag = rank_num - 1

pdb_filename = f"{output_path}/ranked_{tag}.pdb"

show_pdb(pdb_filename, 1, show_sidechains, show_mainchains, color).show()
if color == "lDDT":
  plot_plddt_legend().show()

In [ ]:
# @title Plot plDDT & PAE { display-mode: "form" }
with open(f"{output_path}/ranking_debug.json", 'r') as f:
    data = json.load(f)

plddts = []
paes = []
for file in data['order']:
    pkl_f = bz2.BZ2File(f"{output_path}/result_{file}.pkl.bz2", "r")
    model_data = pickle.load(pkl_f)
    plddts.append(model_data['plddt'])
    paes.append(model_data['predicted_aligned_error'])

plddts_plot = plot_plddts(plddts, [len(sarscov_monomer.sequence)])
pae_plot = plot_paes(paes, [len(sarscov_monomer.sequence)])

plddts_plot.savefig(f"{output_path}/plddts_plot.png")
pae_plot.savefig(f"{output_path}/pae_plots.png")

In [ ]:
# @title Comparison between the AlphaFold structure and a reference in the active state (PDBID: 3P0G) { display-mode: "form" }
rank_num = 1 #@param ["1", "2", "3", "4", "5"] {type:"raw"}
tag = rank_num - 1

reference_cartoon_color = "green" #@param ["green", "cyan", "magenta", "purple", "white", "orange", "yellow", "blue"]
AF_model_cartoon_color = "orange" #@param ["green", "cyan", "magenta", "purple", "white", "orange", "yellow", "blue"]
ligand_color = "whiteCarbon" #@param ["greenCarbon", "cyanCarbon", "magentaCarbon", "purpleCarbon", "whiteCarbon", "orangeCarbon", "yellowCarbon", "blueCarbon"]
reference_sidechain_color = "cyanCarbon" #@param ["greenCarbon", "cyanCarbon", "magentaCarbon", "purpleCarbon", "whiteCarbon", "orangeCarbon", "yellowCarbon", "blueCarbon"]
AF_model_sidechain_color = "yellowCarbon" #@param ["greenCarbon", "cyanCarbon", "magentaCarbon", "purpleCarbon", "whiteCarbon", "orangeCarbon", "yellowCarbon", "blueCarbon"]

pdb_filename = f"{output_path}/ranked_{tag}.pdb"
aligned_model = pdb_filename.split('.')[0]+'_aligned.pdb'


align_on_ref(pdb_filename, ref_file, 'P0G')
show_aligned(aligned_model, ref_aligned, 'P0G',
             model_cartoon_color = AF_model_cartoon_color,
             ref_cartoon_color = reference_cartoon_color,
             ligand_color = ligand_color,
             model_sidecahin_color = AF_model_sidechain_color,
             ref_sidecahin_color = reference_sidechain_color).show()
plot_chain_legend(model_cartoon_color = AF_model_cartoon_color,
             ref_cartoon_color = reference_cartoon_color,
             ligand_color = ligand_color[:-6],
             model_sidecahin_color = AF_model_sidechain_color[:-6],
             ref_sidecahin_color = reference_sidechain_color[:-6]).show()

In [ ]:
# @title Package and Download results { display-mode: "form" }
results_zip = f"ADRB2_multistate.zip"
os.system(f"zip -r {results_zip} {output_path}")
files.download(results_zip)

## Try it yourself
You can try to input the sequence of a new protein target and see predict it using AlphaFold, by performing some changes to the input like template selection, or obtain a more diverse structure by performing subsampling of the MSA

In [ ]:
# @title Inputs { display-mode: "form" }
#@markdown **Protein name**
protein_name = "UserProtein" # @param {type:"string"}
#@markdown **Sequence**
seq = "" # @param {type:"string"}
seq = seq.replace(" ", "")
user_monomer = MonomericObjectMmseqs2(protein_name, seq)

In [ ]:
# @title Search database { display-mode: "form" }

user_monomer.make_features(DEFAULT_API_SERVER=DEFAULT_API_SERVER,
                                            output_dir=protein_features,
                                            templates_path='mmseqs2',
                                            max_template_date='2050-01-01',
                                            filter_t= [{}, []])

%matplotlib inline
plot_msa_v2(sarscov_monomer.feature_dict)
plt.show()
plt.close()

In [ ]:
# @title Filter templates { display-mode: "form" }
filter_templates = False #@param {type:"boolean"}

selected_templates = "" # @param {type:"string"}
excluded_templates = "" # @param {type:"string"}
#@markdown Indicate the templates by writing the relevant **four** symbols PDBID codes seprated by commas.<br/>ex. *3odu,3oe0,4rws*

if filter_templates:
    user_query = {'subset_pdb': [n.upper() for n in selected_templates.split(',')],
                'excluded_pdb': [n.upper() for n in excluded_templates.split(',')]}
    user_monomer.filter_templates([user_query, None], inplace = True)

In [ ]:
# @title Calculations settings { display-mode: "form" }
only_template_models = False # @param {type: 'boolean'}
use_templates = True # @param {type: 'boolean'}
max_seqs = "default" # @param ["default", "8:16", "16:32"]

if max_seq != 'default':
    max_seq, max_extra_seq = max_seqs.split(':')
    max_seq = int(max_seq)
    max_extra_seq = int(max_extra_seq)
    run_description = f'_MSA-subsampling-{max_seqs}'
else:
    max_seq = None
    max_extra_seq = None
    run_description = ''

from litaf.utils import read_custom
    
logging.basicConfig(format="%(levelname)s - %(message)s",
                    level = logging.INFO,
                    handlers=[
                    logging.FileHandler(f"{protein_structures}/{protein_name}.log"),
                    logging.StreamHandler(),
                    ])

n = 2 if only_template_models else 5
if not all([os.path.isfile(f"/params/params_model_{i}_ptm.npz") for i in range(1,6)]):
    download_alphafold_params(f'alphafold2_ptm', Path("."))
model_runners = create_colabfold_runners(
                    '_ptm',
                    n,
                    use_templates,
                    5,
                    ".",
                    max_seq,
                    max_extra_seq,
                    1,
                    False,
                    False,
                    False)

In [ ]:
# @title Run predictions { display-mode: "form" }
output_path = os.path.join(protein_structures, user_monomer.description+run_description)
Path(output_path).mkdir(parents=True, exist_ok=True)
user_monomer.input_seqs = [user_monomer.sequence]
predict(
        model_runners,
        output_path,
        user_monomer.feature_dict,
        0,
        False,
        fasta_name=user_monomer.description+run_description,
        models_to_relax=ModelsToRelax.NONE,
        seqs=[user_monomer.sequence],
        allow_resume=True
    )

In [ ]:
# @title Display 3D structure { display-mode: "form" }
rank_num = 1 #@param ["1", "2", "3", "4", "5"] {type:"raw"}
color = "lDDT" #@param ["chain", "lDDT", "rainbow"]
show_sidechains = False #@param {type:"boolean"}
show_mainchains = False #@param {type:"boolean"}
tag = rank_num - 1

if only_template_models and rank_num > 2:
    print("The structure does not exist, select a rank_num less than 3")
else:
    pdb_filename = f"{output_path}/ranked_{tag}.pdb"
    
    show_pdb(pdb_filename, 1, show_sidechains, show_mainchains, color).show()
    if color == "lDDT":
      plot_plddt_legend().show()

In [ ]:
# @title Plot plDDT & PAE { display-mode: "form" }
with open(f"{output_path}/ranking_debug.json", 'r') as f:
    data = json.load(f)

plddts = []
paes = []
for file in data['order']:
    pkl_f = bz2.BZ2File(f"{output_path}/result_{file}.pkl.bz2", "r")
    model_data = pickle.load(pkl_f)
    plddts.append(model_data['plddt'])
    paes.append(model_data['predicted_aligned_error'])

plddts_plot = plot_plddts(plddts, [len(sarscov_monomer.sequence)])
pae_plot = plot_paes(paes, [len(sarscov_monomer.sequence)])

plddts_plot.savefig(f"{output_path}/plddts_plot.png")
pae_plot.savefig(f"{output_path}/pae_plots.png")

In [ ]:
# @title Package and Download results { display-mode: "form" }
results_zip = f"{protein_name}.zip"
os.system(f"zip -r {results_zip} {output_path}")
files.download(results_zip)